In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd


mypath = "homework/data"
activities = listdir(mypath)
activities

['idle', 'running', 'stairs', 'walking']

In [2]:
for act in activities: 
    path = join(mypath, act)
    frames = listdir(path)
    print(act,":", len(frames))

idle : 1039
running : 3408
stairs : 165
walking : 1850


In [3]:
frames = listdir(join(mypath, 'walking'))
frame = pd.read_csv(join(join(mypath, 'walking'),frames[5]))
frame

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.352658,-9.327810,0.713472
1,-3.955221,-16.496050,3.438074
2,-2.705448,-7.436390,1.383849
3,-4.577713,-17.645266,1.944092
4,7.192182,-25.201366,-16.903065
5,1.575385,-10.620679,1.589750
6,-0.028730,-5.425261,0.823605
7,3.418920,-6.675034,1.158794
8,1.891419,-11.669339,-1.570596
9,-1.647211,-5.305551,9.169792


In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.fftpack import fft, rfft
from sklearn import preprocessing

In [5]:



all_data = []

for activity in activities:
    activity_path = join(mypath, activity)
    files_in_activity = listdir(activity_path)
    

    for file in files_in_activity:
        file_path = join(activity_path, file)
        
 
        df = pd.read_csv(file_path)
        
    
        df['activity'] = activity
        

        all_data.append(df)


In [6]:
# Concatenate all dataframes to create one large dataset
merged_data = pd.concat(all_data, ignore_index=True)
merged_data.head()

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,1.000776,4.616021,8.576031,idle
1,0.718261,4.209007,8.446744,idle
2,-0.909797,-0.282516,9.203311,idle
3,5.099650,0.148441,8.418014,idle
4,1.762132,-0.162806,9.251195,idle


In [7]:

merged_data['min'] = merged_data.drop(columns='activity').min(axis=1)
merged_data['max'] = merged_data.drop(columns='activity').max(axis=1)
merged_data['std'] = merged_data.drop(columns='activity').std(axis=1)
merged_data['mean'] = merged_data.drop(columns='activity').mean(axis=1)


merged_data.head()


,accelerometer_X,accelerometer_Y,accelerometer_Z,activity,min,max,std,mean
0,1.000776,4.616021,8.576031,idle,1.000776,8.576031,3.788412,4.593008
1,0.718261,4.209007,8.446744,idle,0.718261,8.446744,3.867850,4.401144
2,-0.909797,-0.282516,9.203311,idle,-0.909797,9.203311,5.430693,3.622534
3,5.099650,0.148441,8.418014,idle,0.148441,8.418014,4.150876,4.397239
4,1.762132,-0.162806,9.251195,idle,-0.162806,9.251195,4.868658,4.134595


In [8]:
from sklearn.model_selection import train_test_split


X = merged_data.drop(columns=['activity'])
y = merged_data['activity']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


svm = make_pipeline(StandardScaler(), SVC())
svm.fit(X_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [10]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)


RandomForestClassifier()

In [11]:
from sklearn.metrics import accuracy_score

# Predictions
svm_preds = svm.predict(X_test)
rf_preds = rf.predict(X_test)

# Accuracy
svm_acc = accuracy_score(y_test, svm_preds)
rf_acc = accuracy_score(y_test, rf_preds)

print(f"SVM Accuracy: {svm_acc}")
print(f"Random Forest Accuracy: {rf_acc}")


SVM Accuracy: 0.8953884246363355
Random Forest Accuracy: 0.9996389146806974


In [12]:
from sklearn.metrics import classification_report

# For SVM
svm_report = classification_report(y_test, svm_preds)
print("SVM Classification Report:")
print(svm_report)

# For Random Forest
rf_report = classification_report(y_test, rf_preds)
print("Random Forest Classification Report:")
print(rf_report)


SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.96      0.98      0.97      6203
     running       0.93      0.91      0.92     20485
      stairs       1.00      0.00      0.00      1022
     walking       0.81      0.90      0.85     11062

    accuracy                           0.90     38772
   macro avg       0.92      0.70      0.69     38772
weighted avg       0.90      0.90      0.88     38772

Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6203
     running       1.00      1.00      1.00     20485
      stairs       1.00      0.99      0.99      1022
     walking       1.00      1.00      1.00     11062

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772



Судячим з отриманих результатів для SVM було складно виявляти ходьбу по сходам. Однак Random Forest показує хороші результати для всіх активностей.